In [2]:
import numpy as np
import pickle
import glob
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


In [2]:
def load_processed_data(file_path):
    with open(file_path, 'rb') as file:
        X, Y = pickle.load(file)
    return X, Y
X1, Y1 = load_processed_data('./pickel_files/all_data_200_100.pkl')
X1 = np.array(X1)
Y1 = np.array(Y1)
Y1 = Y1[:, :2]


In [3]:
import numpy as np

# Assuming combined_data is your combined dictionary with 'X' and 'Y' keys
combined_data = {'X': [], 'Y': []}
combined_data['X'].extend(X1)
combined_data['Y'].extend(Y1)

import numpy as np
import pickle
import glob

def process_and_combine_pkl_files(directory_path):
    
    # Step 1: Find all .pkl files in the specified directory
    for file_path in glob.glob(directory_path + '/*.pkl'):
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        
        # Step 2: Append the data to combined_data if both 'X' and 'Y' are present
        if 'X' in data and 'Y' in data:
            combined_data['X'].extend(data['X'])
            combined_data['Y'].extend(data['Y'])
    

    
    # Step 4: Convert lists to NumPy arrays
    X_filtered = np.array(combined_data['X'])
    Y_filtered = np.array(combined_data['Y'])
    
    # Further processing of Y (if needed)
    if Y_filtered.shape[1] > 2:
        Y_filtered = Y_filtered[:, :2]

    # Convert X to float 16
    X_filtered = X_filtered.astype(np.float16)
    
    # Convert Y to float 16 
    Y_filtered = Y_filtered.astype(np.float16)
    
    return X_filtered, Y_filtered

# Example usage
directory_path = './process_MPIIGaze'  # Adjust this path as necessary
X_filtered, Y_filtered = process_and_combine_pkl_files(directory_path)


In [8]:
#save X and Y
with open('60K', 'wb') as file:
    pickle.dump([X_filtered, Y_filtered], file)

In [3]:
#load X and Y
with open('60K', 'rb') as file:
    X_filtered, Y_filtered = pickle.load(file)

In [10]:
import numpy as np

def batch_generator(X, Y, batch_size=32, shuffle=True):
    """
    A generator that yields batches of data indefinitely, with optional shuffling.
    """
    num_samples = len(X)
    while True:  # Loop indefinitely
        if shuffle:
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X = X[indices]
            Y = Y[indices]

        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            yield X[start_idx:end_idx], Y[start_idx:end_idx]


In [8]:
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.regularizers import l2
# Maxpooling
from keras.layers import MaxPooling2D
#BatchNormalization
from keras.layers import BatchNormalization

# Your model definition
model = Sequential([
    Conv2D(32, (7, 7), activation='relu', input_shape=(100, 200, 3), kernel_regularizer=l2(0.001)),
    
    
    Conv2D(64, (7, 7), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.15),


    Conv2D(128, (5, 5), activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.15),
    MaxPooling2D((2, 2)),
    

    Conv2D(256, (5, 5), activation='relu'),


    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)



In [5]:
def split_data(X, Y, test_size=0.2, val_size=0.2):
    """
    Splits data into training, validation, and test sets.
    
    Parameters:
    - X, Y: The features and labels.
    - test_size: Fraction of the dataset to be used as the test set.
    - val_size: Fraction of the dataset (after removing the test set) to be used as the validation set.
    
    Returns:
    - X_train, Y_train: Training set.
    - X_val, Y_val: Validation set.
    - X_test, Y_test: Test set.
    """
    # Shuffle the dataset
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    Y = Y[indices]
    
    # Calculate split indices
    test_split_idx = int(len(X) * (1 - test_size))
    val_split_idx = int(test_split_idx * (1 - val_size))
    
    # Split the data
    X_train, X_val, X_test = X[:val_split_idx], X[val_split_idx:test_split_idx], X[test_split_idx:]
    Y_train, Y_val, Y_test = Y[:val_split_idx], Y[val_split_idx:test_split_idx], Y[test_split_idx:]
    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

# Apply the function to your data
X_train, Y_train, X_val, Y_val, X_test, Y_test = split_data(X_filtered, Y_filtered, test_size=0.2, val_size=0.25)


In [6]:
batch_size = 32

num_train_samples = int(len(X_filtered) * 0.6)  # Assuming 60% of data is for training
num_val_samples = int(len(X_filtered) * 0.2)  # Assuming 20% of data is for validation

steps_per_epoch = num_train_samples // batch_size
validation_steps = num_val_samples // batch_size


In [11]:

model.fit(
    x=batch_generator(X_train, Y_train, batch_size=batch_size, shuffle=True),
    steps_per_epoch=steps_per_epoch,
    epochs=100,  # Number of epochs to train for
    validation_data=batch_generator(X_val, Y_val, batch_size=batch_size, shuffle=False),
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)


Epoch 1/100
1242/1242 [==============================] - 253s 192ms/step - loss: 0.0421 - mean_squared_error: 0.0375 - mean_absolute_error: 0.1471 - val_loss: 0.0272 - val_mean_squared_error: 0.0230 - val_mean_absolute_error: 0.1162
Epoch 2/100
1242/1242 [==============================] - 216s 173ms/step - loss: 0.0258 - mean_squared_error: 0.0218 - mean_absolute_error: 0.1113 - val_loss: 0.0165 - val_mean_squared_error: 0.0126 - val_mean_absolute_error: 0.0816
Epoch 3/100
1242/1242 [==============================] - 218s 176ms/step - loss: 0.0221 - mean_squared_error: 0.0183 - mean_absolute_error: 0.1008 - val_loss: 0.0138 - val_mean_squared_error: 0.0102 - val_mean_absolute_error: 0.0722
Epoch 4/100
1242/1242 [==============================] - 214s 172ms/step - loss: 0.0198 - mean_squared_error: 0.0162 - mean_absolute_error: 0.0939 - val_loss: 0.0130 - val_mean_squared_error: 0.0096 - val_mean_absolute_error: 0.0694
Epoch 5/100
1242/1242 [==============================] - 210s 169ms/

In [14]:
#evaluate the model
model.evaluate(X_test, Y_test)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [14]:
from keras.models import load_model
model = load_model('./models/eye_gaze_v18.h5')

In [12]:
#plot predicted vs actual on test data on a canvas using opencv 

import cv2
import numpy as np 
predictions = model.predict(X_test)
screen_width, screen_height = 2650, 1440
canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
cv2.namedWindow('Gaze Tracking on Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking on Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# plot the first 10 images one by one
for i in range(0,25):
    
    # get the predicted x,y coordinates
    x, y = predictions[i][0] * screen_width, predictions[i][1] * screen_height

    # lock the preds 
    x = min(max(x, 0), screen_width)
    y = min(max(y, 0), screen_height)

    # get the actual x,y coordinates
    x_actual, y_actual = Y_test[i][0] * screen_width, Y_test[i][1] * screen_height

    # plot the predicted x,y coordinates
    cv2.circle(canvas, (int(x), int(y)), 10, (0, 0, 255), -1)
    # plot the actual x,y coordinates
    cv2.circle(canvas, (int(x_actual), int(y_actual)), 10, (0, 255, 0), -1 )
    # show the canvas
    cv2.imshow('Gaze Tracking on Canvas', canvas)
    cv2.waitKey(0)
    # # show the image 
    # cv2.imshow('image', X_test[i])
    # cv2.waitKey(0)
    # # clear the canvas
    
    canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)

cv2.waitKey(0)
cv2.destroyAllWindows()

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [13]:
model.save('./models/eye_gaze_v21.h5')